# Feature Engineering and Preprocessing Overview

This notebook combines feature engineering and data preprocessing for customer datasets, designed to optimize memory usage and prepare the data for deep learning. Leveraging **RAPIDS cuDF** and **cuPy**, it processes data in chunks and organizes it for efficient handling in memory-constrained environments.

### **References**
For more discussions on feature engineering and preprocessing:
- [Kaggle Discussion 1](https://www.kaggle.com/competitions/amex-default-prediction/discussion/327828)
- [Kaggle Discussion 2](https://www.kaggle.com/competitions/amex-default-prediction/discussion/328054)

Discussions about data preprocessing are [here][1] and [here][2]

[Kaggle Discussion 3](https://www.kaggle.com/competitions/amex-default-prediction/discussion/327828)
[Kaggle Discussion 4](https://www.kaggle.com/competitions/amex-default-prediction/discussion/328054)

Check the notebook where we got the ideas:
[kaggle Notebook](https://www.kaggle.com/code/cdeotte/tensorflow-gru-starter-0-790)

In [6]:
PROC_DATA = True

# PreProcessing and Chunking

In [3]:
import os
import gc
import cudf
import pandas as pd
import numpy as np
import cupy


def get_row_boundaries(total_rows: int, num_files: int, verbose: str = '') -> list:
    if num_files <= 0:
        raise ValueError("num_files must be a positive integer.")
    
    chunk_size = total_rows // num_files
    boundaries = [(k * chunk_size, (k + 1) * chunk_size - 1 if k != num_files - 1 else total_rows - 1) 
                  for k in range(num_files)]
    
    if verbose:
        print(f'Splitting {verbose} data into {num_files} files:')
        for i, (start, end) in enumerate(boundaries, 1):
            print(f'  Chunk {i}: {start} to {end}')

    return boundaries

def save_boundaries_to_npy(train_boundaries, test_boundaries, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    np.save(f"{output_dir}/train_boundaries.npy", train_boundaries)
    np.save(f"{output_dir}/test_boundaries.npy", test_boundaries)
    print(f"Boundaries saved to {output_dir} as .npy files.")

def load_boundaries_from_npy(output_dir):
    train_boundaries = np.load(f"{output_dir}/train_boundaries.npy", allow_pickle=True)
    test_boundaries = np.load(f"{output_dir}/test_boundaries.npy", allow_pickle=True)
    print(f"Boundaries loaded from {output_dir}.")
    return train_boundaries, test_boundaries

def process_train_test_data(process_data: bool, 
                            path_to_customer_hashes: str = None, 
                            train_num_files: int = 10,
                            test_num_files: int = 5,
                            output_dir: str = './boundaries'):
    train_file_path = f"{output_dir}/train_boundaries.npy"
    test_file_path = f"{output_dir}/test_boundaries.npy"

    if os.path.exists(train_file_path) and os.path.exists(test_file_path):
        print("Boundary files already exist. Loading boundaries.")
        return load_boundaries_from_npy(output_dir)
    else:
        print("Boundary files not found. Processing data to generate boundaries.")

        targets = cudf.read_csv('../input/amex-default-prediction/train_labels.csv')
        targets['customer_ID'] = (targets['customer_ID'].str[-16:]
                                  .str.hex_to_int()
                                  .astype('int64'))
        print(f'Train targets loaded: {targets.shape[0]} rows')
        del targets
        gc.collect()

        if path_to_customer_hashes:
            train = cudf.read_parquet(f'{path_to_customer_hashes}train_customer_hashes.pqt').to_pandas()
        else:
            train = pd.read_csv('/kaggle/input/amex-default-prediction/train_data.csv', 
                                usecols=['customer_ID'], dtype=str, memory_map=True)
        
        print(f'Train data loaded: {train.shape[0]} rows')
        train_boundaries = get_row_boundaries(train.shape[0], num_files=train_num_files, verbose='train')
        del train
        gc.collect()

        test = pd.read_csv('/kaggle/input/amex-default-prediction/test_data.csv', 
                           usecols=['customer_ID'], dtype=str, memory_map=True)
        
        print(f'Test data loaded: {test.shape[0]} rows')
        test_boundaries = get_row_boundaries(test.shape[0], num_files=test_num_files, verbose='test')
        del test
        gc.collect()

        save_boundaries_to_npy(train_boundaries, test_boundaries, output_dir)
        return train_boundaries, test_boundaries

PROCESS_DATA = True
PATH_TO_CUSTOMER_HASHES = None
TRAIN_NUM_FILES = 10
TEST_NUM_FILES = 20
OUTPUT_DIR = './boundaries'

train_boundaries, test_boundaries = process_train_test_data(
    process_data=PROCESS_DATA, 
    path_to_customer_hashes=PATH_TO_CUSTOMER_HASHES,
    train_num_files=TRAIN_NUM_FILES,
    test_num_files=TEST_NUM_FILES,
    output_dir=OUTPUT_DIR
)


Boundary files not found. Processing data to generate boundaries.
Train targets loaded: 458913 rows
Train data loaded: 5531451 rows
Splitting train data into 10 files:
  Chunk 1: 0 to 553144
  Chunk 2: 553145 to 1106289
  Chunk 3: 1106290 to 1659434
  Chunk 4: 1659435 to 2212579
  Chunk 5: 2212580 to 2765724
  Chunk 6: 2765725 to 3318869
  Chunk 7: 3318870 to 3872014
  Chunk 8: 3872015 to 4425159
  Chunk 9: 4425160 to 4978304
  Chunk 10: 4978305 to 5531450
Test data loaded: 11363762 rows
Splitting test data into 20 files:
  Chunk 1: 0 to 568187
  Chunk 2: 568188 to 1136375
  Chunk 3: 1136376 to 1704563
  Chunk 4: 1704564 to 2272751
  Chunk 5: 2272752 to 2840939
  Chunk 6: 2840940 to 3409127
  Chunk 7: 3409128 to 3977315
  Chunk 8: 3977316 to 4545503
  Chunk 9: 4545504 to 5113691
  Chunk 10: 5113692 to 5681879
  Chunk 11: 5681880 to 6250067
  Chunk 12: 6250068 to 6818255
  Chunk 13: 6818256 to 7386443
  Chunk 14: 7386444 to 7954631
  Chunk 15: 7954632 to 8522819
  Chunk 16: 8522820 to 9

In [4]:
print(train_boundaries) 

print(test_boundaries)

[(0, 553144), (553145, 1106289), (1106290, 1659434), (1659435, 2212579), (2212580, 2765724), (2765725, 3318869), (3318870, 3872014), (3872015, 4425159), (4425160, 4978304), (4978305, 5531450)]
[(0, 568187), (568188, 1136375), (1136376, 1704563), (1704564, 2272751), (2272752, 2840939), (2840940, 3409127), (3409128, 3977315), (3977316, 4545503), (4545504, 5113691), (5113692, 5681879), (5681880, 6250067), (6250068, 6818255), (6818256, 7386443), (7386444, 7954631), (7954632, 8522819), (8522820, 9091007), (9091008, 9659195), (9659196, 10227383), (10227384, 10795571), (10795572, 11363761)]


# Feature Engineering 


In [5]:


D_63_MAP = {'CL': 2, 'CO': 3, 'CR': 4, 'XL': 5, 'XM': 6, 'XZ': 7}
D_64_MAP = {'-1': 2, 'O': 3, 'R': 4, 'U': 5}

CAT_OFFSETS = {
    'B_30': 2, 'B_38': 1, 'D_114': 2, 'D_116': 2, 'D_117': 3,
    'D_120': 2, 'D_126': 3, 'D_66': 2,  'D_68': 2
}
CATS_ADDITIONAL = ['D_63', 'D_64']

DTYPE_SKIP = {'customer_ID', 'year', 'month', 'day'}

def _extract_date_parts(df, date_col='S_2'):
    df[date_col] = cudf.to_datetime(df[date_col])
    df['year'] = (df[date_col].dt.year - 2000).astype('int8')
    df['month'] = df[date_col].dt.month.astype('int8')
    df['day'] = df[date_col].dt.day.astype('int8')
    del df[date_col]

def _encode_categoricals(df):
    df['D_63'] = df['D_63'].map(D_63_MAP).fillna(1).astype('int8')
    df['D_64'] = df['D_64'].map(D_64_MAP).fillna(1).astype('int8')

    for c, offset in CAT_OFFSETS.items():
        df[c] = (df[c] + offset).fillna(1).astype('int8')

    return list(CAT_OFFSETS.keys()) + CATS_ADDITIONAL

def _downcast_dtypes(df):
    for c in df.columns:
        if c in DTYPE_SKIP:
            continue
        dtype_str = str(df[c].dtype)
        if dtype_str == 'int64':
            df[c] = df[c].astype('int32')
        elif dtype_str == 'float64':
            df[c] = df[c].astype('float32')

def _pad_rows(df, cats, desired_length=13):
    counts = df.groupby('customer_ID').size()
    if (counts < desired_length).any():
        needed_rows_all = []
        for j in range(1, desired_length):
            idx = counts.index[counts == j]
            if len(idx) > 0:
                needed = cupy.repeat(idx.values, desired_length - j)
                needed_rows_all.append(needed)

        if needed_rows_all:
            needed_rows_all = cupy.concatenate(needed_rows_all)

            pad_df = df.head(len(needed_rows_all)).copy()
            pad_df = pad_df * 0 - 1
            pad_df[cats] = 0
            pad_df['customer_ID'] = needed_rows_all
            df = cudf.concat([df, pad_df], axis=0, ignore_index=True)

    return df

def _final_column_ordering(df, cats):
    cols = list(df.columns)
    cols.remove('customer_ID')
    non_cats = [c for c in cols if c not in cats]
    final_cols = ['customer_ID'] + cats + non_cats
    return df[final_cols]

def feature_engineer(df, PAD_CUSTOMER_TO_13_ROWS=True, targets=None):
    df['customer_ID'] = df['customer_ID'].str[-16:].str.hex_to_int().astype('int64')

    _extract_date_parts(df)

    cats = _encode_categoricals(df)

    _downcast_dtypes(df)

    if PAD_CUSTOMER_TO_13_ROWS:
        df = _pad_rows(df, cats)

    if targets is not None:
        df = df.merge(targets, on='customer_ID', how='left')
        df['target'] = df['target'].astype('int8')

    df = df.fillna(-0.5)

    df = df.sort_values(['customer_ID', 'year', 'month', 'day']).reset_index(drop=True)
    df = df.drop(['year', 'month', 'day'], axis=1)

    df = _final_column_ordering(df, cats)

    return df


In [ ]:
TRAIN_DATA_PATH = '../input/amex-default-prediction/train_data.csv'
TEST_DATA_PATH = '../input/amex-default-prediction/test_data.csv'
TRAIN_LABELS_PATH = '../input/amex-default-prediction/train_labels.csv'
OUTPUT_PATH = './processed_data'

os.makedirs(OUTPUT_PATH, exist_ok=True)

def load_column_names(csv_path):
    temp_df = cudf.read_csv(csv_path, nrows=1)
    return temp_df.columns

def load_targets(csv_path):
    targets = cudf.read_csv(csv_path)
    targets['customer_ID'] = targets['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
    return targets

def process_chunk(csv_path, boundaries, column_names, is_train=True, targets=None, output_prefix='data'):
    for k, (start_idx, end_idx) in enumerate(boundaries, start=1):
        num_rows = end_idx - start_idx + 1

        df = cudf.read_csv(
            csv_path,
            nrows=num_rows,
            skiprows=start_idx + 1,
            header=None,
            names=column_names
        )

        df = feature_engineer(df, targets=targets if is_train else None)

        unique_customers = df['customer_ID'].nunique()
        print(f"{'Train' if is_train else 'Test'}_File_{k} has {unique_customers} customers and shape {df.shape}")

        if is_train:
            tar = df[['customer_ID', 'target']].drop_duplicates().sort_index()
            tar_output_path = os.path.join(OUTPUT_PATH, f'targets_{k}.pqt')
            tar.to_parquet(tar_output_path, index=False)

        if is_train:
            arr = df.iloc[:, 1:-1].values.reshape((-1, 13, 188)).astype('float32')
        else:
            arr = df.iloc[:, 1:].values.reshape((-1, 13, 188)).astype('float32')

        data_output_path = os.path.join(OUTPUT_PATH, f'{output_prefix}_{k}')
        cupy.save(data_output_path, arr)

        del df, arr
        if is_train:
            del tar
        gc.collect()

def process_all(train_boundaries, test_boundaries, process_traindata=True, process_testdata=True):
    column_names = load_column_names(TRAIN_DATA_PATH)
    targets = load_targets(TRAIN_LABELS_PATH)

    if process_traindata:
        process_chunk(
            csv_path=TRAIN_DATA_PATH,
            boundaries=train_boundaries,
            column_names=column_names,
            is_train=True,
            targets=targets,
            output_prefix='data'
        )
        del targets
        gc.collect()

    if process_testdata:
        process_chunk(
            csv_path=TEST_DATA_PATH,
            boundaries=test_boundaries,
            column_names=column_names,
            is_train=False,
            targets=None,
            output_prefix='test_data'
        )

if PROC_DATA:
    process_all(train_boundaries, test_boundaries, process_traindata=True, process_testdata=True)


Train_File_1 has 45869 customers and shape (596297, 190)
Train_File_2 has 45918 customers and shape (596934, 190)
Train_File_3 has 45815 customers and shape (595595, 190)
Train_File_4 has 45796 customers and shape (595348, 190)
Train_File_5 has 45985 customers and shape (597805, 190)
Train_File_6 has 45957 customers and shape (597441, 190)
Train_File_7 has 45918 customers and shape (596934, 190)
Train_File_8 has 45892 customers and shape (596596, 190)
Train_File_9 has 45863 customers and shape (596219, 190)
Train_File_10 has 45907 customers and shape (596791, 190)
Test_File_1 has 46251 customers and shape (601263, 189)
Test_File_2 has 46209 customers and shape (600717, 189)
Test_File_3 has 46136 customers and shape (599768, 189)
Test_File_4 has 46257 customers and shape (601341, 189)
Test_File_5 has 46284 customers and shape (601692, 189)
Test_File_6 has 46244 customers and shape (601172, 189)
Test_File_7 has 46236 customers and shape (601068, 189)
Test_File_8 has 46281 customers and s